In [1]:
import numpy as np
import collections
import struct
import k3d
import open3d as o3d

from random import randint
from sklearn.cluster import DBSCAN

def read_next_bytes(fid, num_bytes, format_char_sequence, endian_character="<"):
    data = fid.read(num_bytes)
    return struct.unpack(endian_character + format_char_sequence, data)

def read_points3D_binary(path_to_model_file):
    with open(path_to_model_file, "rb") as fid:
        num_points = read_next_bytes(fid, 8, "Q")[0]

        xyzs = np.empty((num_points, 3))
        rgbs = np.empty((num_points, 3))
        errors = np.empty((num_points, 1))

        for p_id in range(num_points):
            binary_point_line_properties = read_next_bytes(
                fid, num_bytes=43, format_char_sequence="QdddBBBd")
            xyz = np.array(binary_point_line_properties[1:4])
            rgb = np.array(binary_point_line_properties[4:7])
            error = np.array(binary_point_line_properties[7])
            track_length = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q")[0]
            track_elems = read_next_bytes(
                fid, num_bytes=8*track_length,
                format_char_sequence="ii"*track_length)
            xyzs[p_id] = xyz
            rgbs[p_id] = rgb
            errors[p_id] = error
    return xyzs, rgbs, errors

# bin_file = "/scratch/rhm4nj/cral/datasets/tandt/db/drjohnson/sparse/0/points3D.bin"
bin_file = "/scratch/rhm4nj/cral/datasets/tandt/horse/sparse/0/points3D.bin"

xyzs, rgbs, errors = read_points3D_binary(bin_file)
points = np.array(xyzs)
colors = np.array(rgbs)

print(colors.shape)
print(points.shape)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
(160464, 3)
(160464, 3)


In [2]:
colors = (colors * 255).astype(np.uint8)  # Convert from float [0, 1] to int [0, 255]
colors = (colors[:, 0] << 16 | colors[:, 1] << 8 | colors[:, 2]).astype(np.uint32)  # RGB format

plot = k3d.plot()
point_cloud = k3d.points(positions=points.astype(np.float32), 
                         colors=colors, 
                         point_size=0.01)
plot += point_cloud
plot.display()

Output()

In [3]:
def dbscan_cluster(point_cloud, eps=0.5, min_samples=5):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(point_cloud)

    labels = db.labels_

    unique_labels = set(labels)
    if -1 in unique_labels:
        unique_labels.remove(-1)

    clusters = []
    for label in unique_labels:
        cluster_points = point_cloud[labels == label]
        clusters.append(cluster_points)

    return clusters

def ransac_plane_segmentation(point_cloud, distance_threshold=0.01, max_iterations=1000):
    o3d_cloud = o3d.geometry.PointCloud()
    o3d_cloud.points = o3d.utility.Vector3dVector(point_cloud)

    planes = []
    plane_models = []
    while True:
        plane_model, inliers = o3d_cloud.segment_plane(
            distance_threshold=distance_threshold,
            ransac_n=3,
            num_iterations=max_iterations
        )

        if len(inliers) < 50:  # Adjust this threshold as needed
            break

        plane_points = np.asarray(o3d_cloud.points)[inliers]
        planes.append(plane_points)
        plane_models.append(plane_model)  # Store the plane model (ax + by + cz + d = 0)

        # Remove inliers from the point cloud
        o3d_cloud = o3d_cloud.select_by_index(inliers, invert=True)

    return planes, plane_models

def remove_subset_points(cloud1, cloud2):
    matches = np.isin(cloud1[:, None], cloud2).all(axis=2).any(axis=1)
    filtered_cloud1 = cloud1[~matches]
    return filtered_cloud1

def get_planes_parallel_to_floor(planes, plane_models, floor_model, tolerance=0.2):
    # Extract the normal vector of the floor plane
    floor_normal = np.array(floor_model[:3])

    # Find parallel planes
    parallel_planes = []
    for i, model in enumerate(plane_models):
        plane_normal = np.array(model[:3])
        cos_theta = np.dot(plane_normal, floor_normal) / (np.linalg.norm(plane_normal) * np.linalg.norm(floor_normal))

        # Check if planes are parallel within the tolerance
        if abs(abs(cos_theta) - 1) < tolerance:
            parallel_planes.append(planes[i])

    return parallel_planes

print("RANSAC")
plane_clusters, plane_models = ransac_plane_segmentation(points, distance_threshold=0.05, max_iterations=1000)
floor = plane_clusters[0]
floor_model = plane_models[0]  # Assume the first detected plane is the floor

parallel_planes = get_planes_parallel_to_floor(plane_clusters, plane_models, floor_model)
parallel_planes.append(floor)
print("N parallel:", len(parallel_planes))

print("Cleaning")
cloud = points.copy()
for plane in parallel_planes:
    cloud = remove_subset_points(cloud, plane)

print("Clustering")
cloud = points
clusters = dbscan_cluster(cloud, eps=0.1, min_samples=6)

RANSAC
N parallel: 233
Cleaning
Clustering


In [4]:
clusters_sorted = sorted(clusters, key=lambda x: x.shape[0], reverse=True)

plot = k3d.plot()
n = 25

color = randint(0, 0xFFFFFF)
# plot += k3d.points(
#     positions=floor.astype(np.float32), 
#     point_size=0.1,
#     color=color  # Apply the random color
# )

for i, cluster in enumerate(clusters_sorted[:n]):
    color = randint(0, 0xFFFFFF)

    r = (color >> 16) & 0xFF  # Extract the red component
    g = (color >> 8) & 0xFF   # Extract the green component
    b = color & 0xFF          # Extract the blue component
    rgb = (r, g, b)

    print(i, rgb)
    
    plot += k3d.points(
        positions=cluster.astype(np.float32), 
        point_size=0.1,
        color=color  # Apply the random color
    )

plot.display()

0 (121, 160, 145)
1 (214, 209, 13)
2 (238, 236, 3)
3 (45, 234, 111)
4 (130, 168, 102)
5 (15, 80, 37)
6 (120, 207, 145)
7 (189, 122, 132)
8 (236, 251, 236)
9 (18, 238, 33)
10 (149, 16, 84)
11 (144, 255, 97)
12 (81, 221, 100)
13 (238, 89, 170)
14 (166, 22, 75)
15 (217, 7, 15)
16 (236, 189, 217)
17 (175, 16, 231)
18 (130, 55, 98)
19 (193, 110, 153)
20 (37, 95, 110)
21 (28, 148, 17)
22 (250, 252, 160)
23 (86, 42, 124)
24 (143, 156, 167)


Output()

In [5]:
# from scipy.spatial import ConvexHull
# import numpy as np

# points = clusters_sorted[0]
# # points = (points - points.min(axis=0)) / (points.max(axis=0) - points.min(axis=0))

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)

# pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.0)
# pcd, ind = pcd.remove_radius_outlier(nb_points=25, radius=0.25)

# voxel_size = 0.001  # Adjust based on data
# pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
# cleaned_points = np.asarray(pcd.points)

# hull = ConvexHull(cleaned_points)
# # hull = ConvexHull(points)

# def generate_points_inside_hull(hull, num_points=1000):
#     min_bound = np.min(hull.points, axis=0)
#     max_bound = np.max(hull.points, axis=0)
    
#     random_points = np.random.uniform(min_bound, max_bound, (num_points, len(min_bound)))
#     in_hull = np.all(np.dot(random_points, hull.equations[:, :-1].T) + hull.equations[:, -1] <= 0, axis=1)
    
#     points_inside_hull = random_points[in_hull]
#     return points_inside_hull

# points_on_hull = points[hull.vertices]
# points_inside_hull = generate_points_inside_hull(hull)

# point_size = 0.015
# plot = k3d.plot()
# plot += k3d.points(
#     positions=cleaned_points, 
#     point_size=point_size,
#     color=0xCAAFFF
# )
# plot += k3d.points(
#     positions=points_inside_hull, 
#     point_size=point_size,
#     color=0x0000ff
# )
# plot += k3d.points(
#     positions=points_on_hull, 
#     point_size=point_size,
#     color=0xa83232
# )
# plot.display()

In [6]:
import numpy as np
import alphashape
import trimesh
import k3d
from tqdm import tqdm

points = clusters_sorted[0]
scale_factor = (points - points.min(axis=0)) / (points.max(axis=0) - points.min(axis=0))
points = (points - points.min(axis=0)) / (points.max(axis=0) - points.min(axis=0))

max_iters = 3
for i in tqdm(range(max_iters), desc="iter"):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=0.5)
    pcd, ind = pcd.remove_radius_outlier(nb_points=30, radius=0.15)

    alpha = 10
    alpha_shape = alphashape.alphashape(points, alpha)

    if isinstance(alpha_shape, trimesh.Trimesh):
        is_closed = alpha_shape.is_watertight
        if is_closed:
            print("The alpha shape is closed (watertight).")
        else:
            print("The alpha shape is not closed (not watertight).")
    else:
        print("The alpha shape is not a 3D mesh.")

    points = np.concatenate([points, (alpha_shape.vertices)])

plot = k3d.plot()
point_size = 0.005
plot += k3d.points(positions=points.astype(np.float32), point_size=point_size, color=0x0000ff)
plot += k3d.points(positions=(alpha_shape.vertices).astype(np.float32), point_size=point_size, color=0xff00ff)

vertices = np.array(alpha_shape.vertices)
faces = np.array(alpha_shape.faces)
plot += k3d.mesh(vertices=vertices.astype(np.float32),
                    indices=faces.astype(np.uint32),
                    color=0xff0000,
                    opacity=0.5)  # Set opacity for visualization

plot.camera_position = [500, 500, 500]  # Move the camera further back for a zoomed-out view
plot.camera_target = [0, 0, 0]          # Set the target point (center of the view)
plot.camera_up = [0, 1, 0]              # Set the camera's up direction

plot.display()


iter:  33%|███▎      | 1/3 [00:20<00:40, 20.04s/it]

The alpha shape is not closed (not watertight).


iter:  67%|██████▋   | 2/3 [00:40<00:20, 20.07s/it]

The alpha shape is not closed (not watertight).


iter: 100%|██████████| 3/3 [01:00<00:00, 20.07s/it]

The alpha shape is not closed (not watertight).



/scratch/rhm4nj/cral/.venv/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [7]:
import numpy as np
import alphashape
import trimesh
import k3d
from tqdm import tqdm
from scipy.spatial import KDTree

# points = cleaned_points
boundary_points = alpha_shape.sample(10000)  # Adjust the number of points as needed

com = np.mean(points, axis=0)
min_bound = boundary_points.min(axis=0)
max_bound = boundary_points.max(axis=0)

starting_points = 4000
n_inside = 4000  # Number of points to generate
n_outside = 8000  # Number of points to generate
max_iterations = 100  # Maximum number of iterations to move points towards COM
points_inside = []
points_outside = []

for _ in tqdm(range(max_iterations), desc="iter"):
    random_points = np.random.uniform(low=min_bound, high=max_bound, size=(starting_points, 3))
    inside_mask = np.array([alpha_shape.contains(point.reshape(1, 3)) for point in random_points])

    if len(points_inside) >= n_inside:
        break
    else:
        points_inside.extend(random_points[inside_mask.squeeze()].copy())

    if len(points_outside) >= n_outside:
        break
    else:
        points_outside.extend(random_points[~inside_mask.squeeze()].copy())

    print(len(points_inside))

points_inside = np.array(points_inside)
points_outside = np.array(points_outside)
boundary_points = alpha_shape.sample(10000)


iter:   1%|          | 1/100 [00:02<04:32,  2.76s/it]

500


iter:   2%|▏         | 2/100 [00:05<04:33,  2.80s/it]

1014


iter:   3%|▎         | 3/100 [00:08<04:26,  2.75s/it]

1525


iter:   3%|▎         | 3/100 [00:11<05:55,  3.67s/it]


In [8]:
from scipy.spatial import KDTree

# Assuming 'alpha_shape' is a valid mesh and 'cleaned_points' is your input point cloud
# points = cleaned_points
boundary_points = alpha_shape.sample(10000)  # Adjust the number of boundary points as needed
kdtree = KDTree(alpha_shape.vertices)

# Calculate normals for boundary points
com = np.mean(points, axis=0)
normals = []
for point in tqdm(boundary_points, desc="Calculating normals"):
    distances, indices = kdtree.query(point, k=3)  # Use 3 nearest neighbors to estimate normal
    nearest_points = alpha_shape.vertices[indices]

    # Calculate the normal using the cross product of vectors formed by the nearest neighbors
    v1 = nearest_points[1] - nearest_points[0]
    v2 = nearest_points[2] - nearest_points[0]
    normal = np.cross(v1, v2)

    normal /= np.linalg.norm(normal) + 1e-8  # Add a small epsilon to avoid division by zero
    direction_to_com = com - point
    if np.dot(normal, direction_to_com) > 0:
        # Flip the normal if it points towards the COM
        normal = -normal

    normals.append(normal)

normals = np.array(normals) / 10

def offset_boundary_points(boundary_points, com, offset_parameter):
    vectors_from_com = boundary_points - com
    unit_vectors = vectors_from_com / (np.linalg.norm(vectors_from_com, axis=1, keepdims=True) + 1e-8)
    offset_vectors = unit_vectors * offset_parameter
    offset_boundary_points = boundary_points + offset_vectors
    return offset_boundary_points

def calculate_bounds(points):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    z_min, z_max = np.min(points[:, 2]), np.max(points[:, 2])
    
    return [[x_min, x_max], [y_min, y_max], [z_min, z_max]], (np.array([x_min, y_min, z_min]), np.array([x_max, y_max, z_max]))

pts_on_env = offset_boundary_points(boundary_points, com, 0.001)
all_points = np.vstack([boundary_points, points_outside, points_inside, pts_on_env])
bounds, bounds_coords = calculate_bounds(all_points)
bbox_min, bbox_max = bounds_coords

center_for_translation = (bbox_max + bbox_min) / 2
# scale_factor = max(bbox_max - bbox_min) / 2
scale_factor = 1

bounds_coords = (bounds_coords - center_for_translation) / scale_factor
boundary_points = (boundary_points - center_for_translation) / scale_factor
points_outside = (points_outside - center_for_translation) / scale_factor
points_inside = (points_inside - center_for_translation) / scale_factor
pts_on_env = (pts_on_env - center_for_translation) / scale_factor

all_points = np.vstack([boundary_points, points_outside, points_inside, pts_on_env])
bounds, _ = calculate_bounds(all_points)

Calculating normals: 100%|██████████| 10000/10000 [00:00<00:00, 10831.09it/s]


In [9]:
point_size = 0.01
plot = k3d.plot()
# plot += k3d.points(positions=points.astype(np.float32), point_size=point_size, color=0xff0000)

plot += k3d.points(positions=boundary_points.astype(np.float32), point_size=point_size, color=0x0000ff)
# plot += k3d.vectors(boundary_points.astype(np.float32), normals.astype(np.float32), color=0x0000ff, line_width=0.001, head_size=0.02)

plot += k3d.points(positions=points_inside.astype(np.float32), point_size=point_size, color=0xfff000)
plot += k3d.points(positions=points_outside.astype(np.float32), point_size=point_size, color=0x0ffff0)
plot += k3d.points(positions=pts_on_env.astype(np.float32), point_size=point_size, color=0xaaa0ff)

# vertices = np.array(alpha_shape.vertices)
# faces = np.array(alpha_shape.faces)
# plot += k3d.mesh(vertices=vertices.astype(np.float32),
#                     indices=faces.astype(np.uint32),
#                     color=0xff0000,
#                     opacity=0.5)  # Set opacity for visualization

plot.camera_position = [500, 500, 500]  # Move the camera further back for a zoomed-out view
plot.camera_target = [0, 0, 0]          # Set the target point (center of the view)
plot.camera_up = [0, 1, 0]              # Set the camera's up direction

plot.display()

Output()

In [10]:
## Save points to file
## transpose to match the convention of the other code
import os

save_dir = 'tmp_outs'
if not os.path.exists(save_dir): os.makedirs(save_dir)

print('saving bounds:', bounds)
np.save(f'{save_dir}/bounds.npy', bounds)

print('saving scale_factor.npy:', scale_factor)
np.save(f'{save_dir}/scale_factor.npy', scale_factor)

print('saving env_outside_pts:', points_outside.shape)
np.save(f'{save_dir}/env_outside_pts.npy', points_outside)

print('saving pts_inside:', points_inside.shape)
np.save(f'{save_dir}/pts_inside.npy', points_inside)

print('saving pts_on_env:', pts_on_env.shape)
np.save(f'{save_dir}/pts_on_env.npy', pts_on_env)

print('saving interface_pts:', boundary_points.shape)
np.save(f"{save_dir}/interface_pts.npy", boundary_points)

print('saving interface_normals:', normals.shape)
np.save(f"{save_dir}/interface_normals.npy", normals)

# print('saving pts_around_interface_outside_env:', pts_around_interface_outside_env.shape)
# np.save(f"{save_dir}/pts_around_interface_outside_env.npy", pts_around_interface_outside_env)

saving bounds: [[-0.49460269793817585, 0.49460269793817585], [-0.4988967760598918, 0.4988967760598918], [-0.4964279473317935, 0.49642794733179346]]
saving scale_factor.npy: 1
saving env_outside_pts: (10475, 3)
saving pts_inside: (2029, 3)
saving pts_on_env: (10000, 3)
saving interface_pts: (10000, 3)
saving interface_normals: (10000, 3)


In [11]:
# import open3d as o3d
# import numpy as np
# import k3d

# # Example point cloud (replace with your actual data)
# points = cleaned_points * 100  # Replace with your point cloud data
# mypoints = points.copy()
# voxel_size = 5  # Adjust voxel size as needed
# alph = .75

# # Calculate the center of mass (COM)
# com = np.mean(points, axis=0)

# max_iters = 10
# for i in range(max_iters):
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(points)
#     voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)

#     interior_points = []
#     outer_points = []
#     for voxel in voxel_grid.get_voxels():
#         voxel_center = voxel.grid_index * voxel_size + voxel_grid.origin

#         corners = [
#             voxel_center + np.array([dx, dy, dz]) * (voxel_size / 2)
#             for dx in [-1, 1]
#             for dy in [-1, 1]
#             for dz in [-1, 1]
#         ]

#         distances = [np.linalg.norm(corner - com) for corner in corners]
#         closest_corner = corners[np.argmin(distances)]

#         interior_points.append(closest_corner)

#     interior_points = np.array(interior_points)
#     points = np.concatenate([points, interior_points])
#     voxel_size *= alph

# # Visualization using k3d
# plot = k3d.plot()
# point_size = 1
# plot += k3d.points(positions=mypoints.astype(np.float32), point_size=point_size, color=0x0000ff)  # Original points
# plot += k3d.points(positions=interior_points.astype(np.float32), point_size=point_size, color=0x00ff00)  # Interior points
# plot += k3d.points(positions=(com.reshape(1, -1)).astype(np.float32), point_size=point_size, color=0xffff00)  # Interior points
# plot.display()


In [12]:
# from scipy.spatial import Delaunay
# import numpy as np

# # Example point cloud (replace with your data)
# points = cleaned_points * 100

# # Create a 3D Delaunay triangulation
# tri = Delaunay(points)

# # Generate random points within the bounding box
# num_points_to_generate = 10000
# min_bound = points.min(axis=0)
# max_bound = points.max(axis=0)
# random_points = np.random.uniform(low=min_bound, high=max_bound, size=(num_points_to_generate, 3))

# # Check if random points are inside the Delaunay triangulation
# is_inside = tri.find_simplex(random_points) >= 0
# interior_points = random_points[is_inside]

# # Visualization with k3d (optional)
# plot = k3d.plot()
# point_size = 1
# plot += k3d.points(positions=points.astype(np.float32), point_size=point_size, color=0x0000ff)  # Original points
# plot += k3d.points(positions=interior_points.astype(np.float32), point_size=point_size, color=0x00ff00)  # Interior points
# plot.display()